In [1]:
# 2019.02.11 seminar LSTM part(RNN)
# made by pbj0812
# mnist classification by RNN

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../mnist/data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../mnist/data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../mnist/data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../mnist/data\t10k-images-idx3-ubyte.gz
Extracting ../mnist/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
learning_rate = 0.001
total_epoch = 100
batch_size = 128

In [5]:
global_step = tf.Variable(0, trainable=False, name='global_step')

In [6]:
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10

In [7]:
X = tf.placeholder(tf.float32, [None, n_step, n_input]) # y : 28, x : 28
Y = tf.placeholder(tf.float32, [None, n_class]) # 10 class (0~9)

In [8]:
with tf.name_scope('layer1'):
    W = tf.Variable(tf.random_normal([n_hidden, n_class]), name='W') # 128 layer, 10 class
    b = tf.Variable(tf.random_normal([n_class])) # 10 class

with tf.name_scope('output'):
    cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden) # create RNN cell that have n_hidden(output)
    outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    # outputs : [batch_size, n_step, n_hidden]
    outputs = tf.transpose(outputs, [1, 0, 2]) # [n_step, batch_size, n_hidden]
    outputs = outputs[-1] # [batch_size, n_hidden]
    model = tf.matmul(outputs, W) + b

In [9]:
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
    # global_step!!!
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost, global_step=global_step)
    tf.summary.scalar('cost', cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [10]:
# train

In [11]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [12]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

In [13]:
total_batch = int(mnist.train.num_examples / batch_size)

In [14]:
for epoch in range(total_epoch):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        
        _, cost_val = sess.run([optimizer, cost],
                              feed_dict = {X:batch_xs, Y:batch_ys})
        total_cost += cost_val
        
    print('Epoch : ', '%04d' % (epoch + 1),
         'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
    
    summary = sess.run(merged, feed_dict={X:batch_xs, Y:batch_ys})
    writer.add_summary(summary, global_step=sess.run(global_step))

saver.save(sess, './model/lstm.ckpt', global_step=global_step)
print('complete!!!')

Epoch :  0001 Avg. cost = 0.447
Epoch :  0002 Avg. cost = 0.131
Epoch :  0003 Avg. cost = 0.090
Epoch :  0004 Avg. cost = 0.067
Epoch :  0005 Avg. cost = 0.056
Epoch :  0006 Avg. cost = 0.046
Epoch :  0007 Avg. cost = 0.039
Epoch :  0008 Avg. cost = 0.036
Epoch :  0009 Avg. cost = 0.032
Epoch :  0010 Avg. cost = 0.029
Epoch :  0011 Avg. cost = 0.026
Epoch :  0012 Avg. cost = 0.021
Epoch :  0013 Avg. cost = 0.022
Epoch :  0014 Avg. cost = 0.021
Epoch :  0015 Avg. cost = 0.019
Epoch :  0016 Avg. cost = 0.016
Epoch :  0017 Avg. cost = 0.014
Epoch :  0018 Avg. cost = 0.013
Epoch :  0019 Avg. cost = 0.016
Epoch :  0020 Avg. cost = 0.013
Epoch :  0021 Avg. cost = 0.013
Epoch :  0022 Avg. cost = 0.010
Epoch :  0023 Avg. cost = 0.010
Epoch :  0024 Avg. cost = 0.011
Epoch :  0025 Avg. cost = 0.009
Epoch :  0026 Avg. cost = 0.006
Epoch :  0027 Avg. cost = 0.009
Epoch :  0028 Avg. cost = 0.011
Epoch :  0029 Avg. cost = 0.007
Epoch :  0030 Avg. cost = 0.006
Epoch :  0031 Avg. cost = 0.008
Epoch : 

In [15]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [16]:
test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

In [17]:
print('accuracy : ', sess.run(accuracy, feed_dict={X:test_xs, Y:test_ys}))

accuracy :  0.9907
